<h1>TME 02 : Inférence Collective</h1>

<h3>Importations des librairies utilisées</h3>

In [1]:
import random 
import numpy as np

<h3>Fonctions de chargement des données</h3>

La fonction loadLabels() s'applique sur un fichier *.content et retourne deux dictionnaires, le premier, labels, contient pour chaque lien son label, et est de la forme { lien1 : label1, lien2 : labels2 .... }. Le deuxieme, attrs, contient pour chaque lien, sa liste d'attributs ( liste de 0 et de 1 ), il est de la forme : {lien1 : [0,1,0,....], lien2 : [1,0,0,....]}

In [78]:
def loadLabels(path='ms/WebKB/content', file='texas.content'):     
    labels={};     
    attrs ={}; 
    for line in open(path+'/'+file): 
        line  = line[:-1]        
        link = line.split('\t') 
        labels.setdefault(link[0],{}) 
        attrs.setdefault(link[0],{}) 
          
        labels[link[0]] = link[len(link)-1] 
        attrs[link[0]] = link[1:len(link)-1] 
    return labels,attrs  
labels, attrs = loadLabels();

La fonction loadLabels() s'applique sur un fichier *.cites, et retourne deux graphes, le premier graphe, graph, contient pour chaque lien, les liens vers lesquels il pointe (successeurs), et le deuxieme graphe, graphInv, contient pour chaque lien, les liens qui pointent vers lui (prédécesseurs).

In [3]:
def loadGraph(path='ms/WebKB/cites', file='texas.cites'):     
    graph    ={};     
    graphInv ={};     
    for line in open(path+'/'+file):             
        line  = line[:-1]  
        (dest,source) = line.split(' ') 
         
        if dest not in graph: 
            graph.setdefault(dest,{})             
            graph[dest] = source 
        else: 
            graph[dest] = graph[dest]+';'+source 
        ############################################     
        if source not in graphInv: 
            graphInv.setdefault(source,{})             
            graphInv[source] = dest 
        else: 
            graphInv[source] = graphInv[source]+';'+dest 
              
    return graph, graphInv 
graph, graphInv = loadGraph();                        

Afin de construire les vecteurs, nous utiliserons ces quelques fonctions de traitement:
La fonction getClasses() retourne les classes des liens voisins du lien donné en paramètre, ainsi que leurs nombres.
La fonction getVecteurLabels() retourne un vecteur de taille 5 (les 5 labels), représentant les proportions des classes. 

In [85]:
def get(links): 
     return links.split(';'); 
def getClasses(link, graphe, grapheInv): 
    classes={} 
    if link in graphe: 
        nexts = get(graphe[link]); 
        for i in nexts: 
            if labels[i] not in classes: 
                classes.setdefault(labels[i],{}) 
                classes[labels[i]] = 1 
            else:     
                classes[labels[i]] = classes[labels[i]] + 1
    if link in grapheInv:     
        precs = get(grapheInv[link]); 
        for i in precs: 
            if labels[i] not in classes: 
                classes.setdefault(labels[i],{}) 
                classes[labels[i]] = 1 
            else:     
                classes[labels[i]] = classes[labels[i]] + 1   
    return classes;  
def getVecteurLabels(classes): 
    #0: course, 1: faculty, 2: student, 3: project, 4: staff         
    v = [0.0, 0.0, 0.0, 0.0, 0.0] 
    size = len(classes) 
    for i in classes: 
        if i == 'course': 
            v[0] = 1.0 / size 
        elif i == 'faculty': 
            v[1] = 1.0 / size 
        elif i == 'student': 
            v[2] = 1.0 / size 
        elif i == 'project': 
            v[3] = 1.0 / size 
        elif i == 'staff': 
            v[4] = 1.0 / size     
    return v;         
def getLabelByNumber(i):
    if i == 1: 
        return 'course'
    elif i == 2: 
        return 'faculty'
    elif i == 3: 
        return 'student'
    elif i == 4: 
        return 'project'
    elif i == 5: 
        return 'staff'  
def getNumberOfLabel(i):
    if i == 'course': 
        return 1
    elif i == 'faculty': 
        return 2
    elif i == 'student': 
        return 3
    elif i == 'project': 
        return 4
    elif i == 'staff': 
        return 5    

La fonction constructVectors() construit les vecteurs de tous les liens du graphe.

In [5]:
def constructVectors(labels, attrs, graphe, grapheInv): 
    vecteurs = {}; 
    for link in labels:         
        vecteurs.setdefault(link,{}) 
        i = [] 
        # Ajouter les liens: 
        i.extend(attrs[link])          
        # Extraire les classes
        classes = getClasses(link, graphe, grapheInv)          
        # Construire le mini-vecteur de classes
        vectLab = getVecteurLabels(classes)          
        # Ajouter les labels des voisins: 
        i.extend(vectLab)           
        vecteurs[link] = i 
    return vecteurs; 
        
vecteurs = constructVectors(labels, attrs, graph, graphInv);   

Nous allons ensuite prendre 10% de ces vecteurs à l'aide de la fonction getPourcentage(), puis à l'aide la fonciton getXY(), nous allons transformer ces vecteurs en matrice.

In [81]:
def getVecteurTrainTest(vecteur, labels, pourcentage):     
    s = pourcentage*len(vecteur) ;           
    t = len(vecteur)- int(s);
    vTrain = {} 
    vTest = {}    
    choosed = []   
    # Train   
    while s>1:             
        link = random.choice(list(vecteur));             
        while link in choosed:
            link = random.choice(list(vecteur));                
        choosed.append(link);       
        vTrain.setdefault(link+','+labels[link],{}) 
        vTrain[link+','+labels[link]] = vecteur[link] 
        s = s-1 
    # Test      
    while t>0:             
        link = random.choice(list(vecteur));             
        while link in choosed:
            link = random.choice(list(vecteur));                
        choosed.append(link);       
        vTest.setdefault(link+','+labels[link],{}) 
        vTest[link+','+labels[link]] = vecteur[link] 
        t = t-1            
    return vTrain,vTest
                            
def getXY(vecteurs,labels,pourcentage):
    vTrain, vTest = getVecteurTrainTest(vecteurs,labels,pourcentage);     
    XTrain = []
    YTrain = []
    XTest  = []
    YTest  = []
    for link in vTrain: 
        XTrain.append(vTrain[link])                 
        l = getNumberOfLabel(link.split(',')[1])
        YTrain.append(l);        
    XTrain = np.array(XTrain)    
    for link in vTest: 
        XTest.append(vTest[link])                 
        l = getNumberOfLabel(link.split(',')[1])
        YTest.append(l);
    XTest = np.array(XTest)
    return XTrain,YTrain,XTest,YTest      

In [7]:
XTrain,YTrain,XTest,YTest = getXY(vecteurs,labels,0.9);
XTrain = np.array(XTrain,dtype=float);
YTrain = np.array(YTrain,dtype=float);
XTest = np.array(XTest,dtype=float);
YTest = np.array(YTest,dtype=float);

La fonction getTrainTestLabels() s'applique aux labels, elle découpe le dictionnaire labels en train et test selon un pourcentage donné en paramètre.

In [8]:
def getTrainTestLabels(labels, pourcentage):
    size = len(labels)
    #tr = pourcentage*len(labels) ;           
    #te = len(labels)- int(tr);
    trainLabels = {}     
    testLabels = {}    
    for link in labels:
        trainLabels.setdefault(link,{})
        if random.random() < pourcentage:
            trainLabels[link] = labels[link]
        else:    
            trainLabels[link] = ''    
    for link in trainLabels:  
        if trainLabels[link]== '':
            testLabels.setdefault(link,{})
            testLabels[link] = labels[link]
    return trainLabels,testLabels 

<b>Nous Allons tester plusieurs modèles pour voir celui qui prédit le mieux</b><br\>
La foret aléatoire:

In [106]:
from sklearn.ensemble import RandomForestClassifier 
rfc = RandomForestClassifier(n_estimators = 200)
rfc.fit(XTrain,YTrain)
output = rfc.predict(XTest);
score = rfc.score(XTest,YTest);
print(score)

0.684210526316


Le modèle linéaire:

In [50]:
from sklearn import linear_model
lin = linear_model.LinearRegression()
lin.fit(XTrain,YTrain)
lin.predict(XTest)
output = lin.predict(XTest)
score  = lin.score(XTest,YTest)
print(score)


0.018360016964


La déscente de gradient stochastique l2:

In [11]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss="hinge", penalty="l2")
sgd.fit(XTrain,YTrain)
output = sgd.predict(XTest)
score  = sgd.score(XTest,YTest)
print(score)

0.789473684211


Le Perceptron

In [12]:
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import PolynomialFeatures
perceptron = Perceptron(fit_intercept=False, n_iter=100, shuffle=False).fit(XTrain, YTrain)
perceptron.score(XTest, YTest)

0.78947368421052633

GradientBoostingClassifier

In [93]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(XTrain,YTrain)
#output = gbc.predict(XTest)
score  = gbc.score(XTest,YTest)
print(score)

0.842105263158


<h3>Classifieur</h3>

In [118]:
def classifieur(noeud, model, vecteurs):
    container = [];
    v = list(map(float, vecteurs[noeud]))    
    container.append(np.array(v))            
    output = model.predict(v)    
    return getLabelByNumber(output[0]);

In [119]:
def updateLabels(y,labels):
    for link in y:
        labels[link] = y[link]
    return labels;    
def getOrdreAleatoire(nombre, y):
    O = [];
    while nombre > 1:
        link = random.choice(list(y));
        O.append(link)
        nombre = nombre -1;
    return O;    

In [120]:
class ICA:
    def __init__(self, model, vecteurs, labels):
        self.model    = model;
        self.vecteurs = vecteurs;
        self.labels   = labels;
   
    def predict(self):
        y = {}
        #bootstraping
        # On infère un label pour chaque noeud de label non connu, un label à partir de ses attributs et les labels 
        # connus des noeuds de son voisinage.
        for noeud in self.labels:        
            if self.labels[noeud] == '':
                y[noeud] = classifieur(noeud, self.model, self.vecteurs)            
        self.labels = updateLabels(y,self.labels)            
        #Tant qu'on n'est pas arrivé à un étiquetage stable:    
        for i in range(1000):
            #1 Tire un ordre aléatoire O sur les noeuds dont le label n'est pas connu
            O = getOrdreAleatoire(20, y)
            #iteration
            for i in range(len(O)):
                y[noeud] = classifieur(O[i], self.model, self.vecteurs)
            #2 Puis on itére sur O en determinant le label de plus fort score pour chaque noeud, à partir de ses 
            #attributs et les labels des noeuds de son voisinage
            self.labels = updateLabels(y,self.labels)         
        return self.labels;   
    def score(self,testLabels):
        count = 0.0
        match = 0.0
        for link in testLabels:
            count = count+1            
            if testLabels[link] == self.labels[link]:
                match=match+1
                #print yc,' - ',labels[link]
        return match/count

In [121]:
labels, attrs = loadLabels();
trainLabels,testLabels = getTrainTestLabels(labels, 0.8);
ica = ICA(perceptron,vecteurs,trainLabels)
labelsPredicted = ica.predict();
print ica.score(testLabels)

1.0


<h3>Propagation de labels</h3>